# Data preparation

In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
import json

SEED = 1234
tf.random.set_seed(SEED)

subset_folder='Bipbip'

subset_file=subset_folder.replace('/','_')+'.json'

test_name = 'U-Net_basic' + subset_folder.replace('/','_')

In [ ]:
# Set the base directory for Colab and non Colab environment
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  base_dir = '/content/drive/My Drive/AN2DL/homework_2'
else:
  base_dir = os.getcwd()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create two different training ImageDataGenerator object for images and corresponding masks
img_data_gen = ImageDataGenerator(rotation_range=10,
                                  width_shift_range=10,
                                  height_shift_range=10,
                                  zoom_range=0.3,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode='reflect',
                                  rescale=1./255)
mask_data_gen = ImageDataGenerator(rotation_range=10,
                                   width_shift_range=10,
                                   height_shift_range=10,
                                   zoom_range=0.3,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='reflect')

# Create validation and test ImageDataGenerator objects
valid_img_data_gen = ImageDataGenerator(rescale=1./255)
valid_mask_data_gen = ImageDataGenerator()

In [ ]:
from PIL import Image

class CustomDataset(tf.keras.utils.Sequence):
  
  RGB_TO_TARGET = [([0, 0, 0], 0),        # background
                   ([216, 124, 18], 0),   # background
                   ([255, 255, 255], 1),  # crop
                   ([216, 67, 82], 2)]    # weed

  def __init__(self, dataset_dir, which_subset, subset_file, subset_folder,
               img_generator=None, mask_generator=None, preprocessing_function=None, out_shape=None):
    if which_subset == 'training':
      subset_dir = os.path.join(dataset_dir, 'Training', subset_folder)
    elif which_subset == 'validation':
      subset_dir = os.path.join(dataset_dir, 'Validation', subset_folder)

    subset_file = os.path.join(subset_dir, subset_file)
    
    with open(subset_file, 'r') as f:
      files = json.load(f)
    

    self.files = files
    self.which_subset = which_subset
    self.dataset_dir = dataset_dir
    self.subset_dir = subset_dir
    self.subset_file = subset_file
    self.img_generator = img_generator
    self.mask_generator = mask_generator
    self.preprocessing_function = preprocessing_function
    self.out_shape = out_shape

  def __len__(self):
    return len(self.files)

  def __getitem__(self, index):
    # Read Image
    img = Image.open(os.path.join(self.subset_dir, 
                                  self.files[index]['path'], 
                                  'Images', self.files[index]['file'] + '.jpg'))
    mask = Image.open(os.path.join(self.subset_dir, 
                                   self.files[index]['path'], 
                                   'Masks', self.files[index]['file'] + '.png'))

    # Resize image and mask
    if self.out_shape is not None:
      img = img.resize(self.out_shape)
      mask = mask.resize(self.out_shape, resample=Image.NEAREST)

    img_arr = np.array(img)
    mask_arr = np.array(mask)

    if self.img_generator is not None and self.mask_generator is not None:
      # Perform data augmentation
      # We can get a random transformation from the ImageDataGenerator using get_random_transform
      # and we can apply it to the image using apply_transform
      img_t = self.img_generator.get_random_transform(img_arr.shape, seed=SEED)
      mask_t = self.mask_generator.get_random_transform(mask_arr.shape, seed=SEED)
      img_arr = self.img_generator.apply_transform(img_arr, img_t)
      # ImageDataGenerator use bilinear interpolation for augmenting the images.
      # Thus, when applied to the masks it will output 'interpolated classes', which
      # is an unwanted behaviour. As a trick, we can transform each class mask 
      # separately and then we can cast to integer values (as in the binary segmentation notebook).
      # Finally, we merge the augmented binary masks to obtain the final segmentation mask.
      out_mask = np.zeros(mask_arr.shape[:2], dtype=mask_arr.dtype)

      for rgb, c in self.RGB_TO_TARGET:
        if c > 0:
          curr_class_arr = np.copy(mask_arr)
          curr_class_arr[np.where(np.all(curr_class_arr != rgb, axis=-1))] = [0, 0, 0]
          curr_class_arr = self.mask_generator.apply_transform(curr_class_arr, mask_t)

          class_out_mask = np.zeros(mask_arr.shape[:2], dtype=mask_arr.dtype)
          class_out_mask[np.where(np.all(curr_class_arr == rgb, axis=-1))] = c

          out_mask += class_out_mask
    else:
      out_mask = mask_arr
    
    if self.preprocessing_function is not None:
        img_arr = self.preprocessing_function(img_arr)

    return img_arr, np.float32(out_mask)

In [ ]:
img_h = 1536
img_w = 2048

dataset_dir = os.path.join(base_dir, 'Development_Dataset')

dataset = CustomDataset(dataset_dir, 'training', 
                        subset_folder=subset_folder,
                        subset_file=subset_file,
                        img_generator=img_data_gen, 
                        mask_generator=mask_data_gen, 
                        out_shape=(img_w, img_h))

dataset_valid = CustomDataset(dataset_dir, 'validation',
                              subset_folder = 'Bipbip',
                              subset_file='Bipbip.json',
                              img_generator=valid_img_data_gen, 
                              mask_generator=valid_mask_data_gen, 
                              out_shape=(img_w, img_h))

In [ ]:
train_dataset = tf.data.Dataset.from_generator(lambda: dataset,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_h, img_w, 3], [img_h, img_w]))

train_dataset = train_dataset.batch(1)

train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: dataset_valid,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_h, img_w, 3], [img_h, img_w]))
valid_dataset = valid_dataset.batch(1)

valid_dataset = valid_dataset.repeat()

# Model

In [ ]:
def conv2D(x, filters, k_size=3, strirdes=1, relu=0, pool_size=0):
    x = tf.keras.layers.Conv2D(filters=filters, 
                               kernel_size=k_size,
                               strides=strirdes,
                               padding='same',
                               input_shape=[None])(x)
    if relu:
        x = tf.keras.layers.ReLU()(x)
    if pool_size:
        x = tf.keras.layers.MaxPool2D(pool_size=pool_size)(x)
    return x

In [ ]:
def upSampling(x, up_type=0, up_size=2, filters=1):
    if up_type == 1:
        x = tf.keras.layers.Conv2DTranspose(filters=filters, 
                                            kernel_size=3,
                                            strides=up_size,
                                            padding='same',
                                            input_shape=[None])(x)
    else:
        x = tf.keras.layers.UpSampling2D(up_size,
                                         interpolation='bilinear')(x)
    return x

In [ ]:
def cropping2D(y, x):
    h_cropp = y.shape[1]-x.shape[1]
    w_cropp = y.shape[2]-x.shape[2]
    if h_cropp%2:
        h_cropp = (h_cropp//2+1, h_cropp//2)
    else:
        h_cropp = (h_cropp//2, h_cropp//2)
    if w_cropp%2:
        w_cropp = (w_cropp//2+1, w_cropp//2)
    else:
        w_cropp = (w_cropp//2, w_cropp//2)

    y = tf.keras.layers.Cropping2D(cropping=(h_cropp, w_cropp))(y)
    return y

In [ ]:
def get_encoder(inputs, deepth, filters):
    skip_passes = []

    x = inputs
    ### [First half of the network: downsampling inputs] ###
    for i in range(deepth):
        k_size = 3
        x = conv2D(x, filters, k_size=k_size, relu=1)
        x = conv2D(x, filters, k_size=k_size, relu=1)
        skip_passes.append(x)
        x = tf.keras.layers.MaxPool2D(pool_size=2)(x)
        filters *= 2

    return x, skip_passes, filters

In [ ]:
def get_UNet(input_shape, num_classe):
    inputs = keras.Input(shape=input_shape)

    deepth = 4
    filters = 32

    x, skip_passes, filters = get_encoder(inputs, deepth, filters)
    
    skip_passes = skip_passes[::-1]
    # Bottleneck
    x = conv2D(x, filters, relu=1)
    x = conv2D(x, filters, relu=1)

    ### Decoder ###    
    for i in range(deepth):
        filters //= 2
        x = upSampling(x, filters)
        y = skip_passes[i]
        x = tf.keras.layers.concatenate([y, x]) 
        x = conv2D(x, filters, relu=1)
        x = conv2D(x, filters, relu=1)

    # Add a per-pixel classification layer
    outputs = tf.keras.layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = tf.keras.Model(inputs, outputs)
    return model



In [ ]:
img_size = (img_h, img_w, 3)
num_classes = 3

# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

model = get_UNet(img_size, num_classes)

# Visualize created model as a table
model.summary()

# Visualize initialized weights
# model.weights

In [ ]:
# Optimization params
# -------------------

# Loss
# Sparse Categorical Crossentropy to use integers (mask) instead of one-hot encoded labels
loss = tf.keras.losses.SparseCategoricalCrossentropy() 
# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

epochs = 100

# Here we define the intersection over union for each class in the batch.
# Then we compute the final iou as the mean over classes
def meanIoU(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.argmax(y_pred, -1)

    per_class_iou = []

    for i in range(1,3): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

# Validation metrics
# ------------------
metrics = [meanIoU]
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
exp_dir = os.path.join(base_dir, test_name)
if not os.path.exists(exp_dir):
  os.makedirs(exp_dir)
    
callbacks = []

# Checkpoint callback, generate a checkpoint at each epoch
# There is only one checkpoint overwritten only if the new one has a lower validation loss
ckpt_dir = os.path.join(exp_dir, 'checkpoints')
if not os.path.exists(ckpt_dir):
  os.makedirs(ckpt_dir)
checkpoint = os.path.join(ckpt_dir, 'checkpoint_' + test_name + '.ckpt')
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint,
                                                   save_weights_only=True,
                                                   monitor='val_meanIoU',
                                                   mode='max',
                                                   save_best_only=True)
callbacks.append(ckpt_callback)

# Checkpoint for tensorboard logs
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
  os.makedirs(tb_dir)
    
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)
callbacks.append(tb_callback)

# Checkpoint for early stopping in order to optimize the number of epochs
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_meanIoU', mode='max', patience=5)
callbacks.append(es_callback)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/My\ Drive/AN2DL/homework_2/

In [ ]:
# If there exists a checkpoint it's loaded, otherwise the model is trained
# If train is True and there exists a checkpoint the trainig continues from it

train = False
load = False

if os.path.exists(os.path.join(ckpt_dir, 'checkpoint')) and load:
  model.load_weights(checkpoint)
else:
  train = True

if train:
  model.fit(x=train_dataset,
            epochs=epochs,
            steps_per_epoch=len(dataset),
            validation_data=valid_dataset,
            validation_steps=len(dataset_valid), 
            callbacks=callbacks)
  
  # Reload the weights to load the best checkpoint
  model.load_weights(checkpoint)

  # Save model
  model_file = os.path.join(exp_dir, test_name + '.h5')
  model.save(model_file, include_optimizer=False)

# Prediction

In [ ]:
import time
import matplotlib.pyplot as plt

from PIL import Image

%matplotlib inline

iterator = iter(valid_dataset)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 30))
fig.show()
image, target = next(iterator)

target = target[0]
out_sigmoid = model.predict(image)
image = image[0]
prediction = tf.argmax(out_sigmoid[0], -1)

# Assign colors (just for visualization)
target_img = np.zeros([target.shape[0], target.shape[1], 3])
prediction_img = np.zeros([prediction.shape[0], prediction.shape[1], 3])

target_img[np.where(target == 0)] = [0, 0, 0]
target_img[np.where(target == 1)] = [0, 255, 0]
target_img[np.where(target == 2)] = [255, 0, 0]

prediction_img[np.where(prediction == 0)] = [0, 0, 0]
prediction_img[np.where(prediction == 1)] = [0, 255, 0]
prediction_img[np.where(prediction == 2)] = [255, 0, 0]

ax[0].imshow(np.uint8(image))
ax[1].imshow(np.uint8(target_img))
ax[2].imshow(np.uint8(prediction_img))

fig.canvas.draw()
time.sleep(1)

In [ ]:
def rle_encode(img):
  
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def add_prediction(img_name, img_shape, mask_arr, team, crop, submission_dict):

  mask_arr = np.array(mask_arr)
  submission_dict[img_name] = {}
  submission_dict[img_name]['shape'] = img_shape
  submission_dict[img_name]['team'] = team
  submission_dict[img_name]['crop'] = crop
  submission_dict[img_name]['segmentation'] = {}

  # RLE encoding
  # crop
  rle_encoded_crop = rle_encode(mask_arr == 1)
  # weed
  rle_encoded_weed = rle_encode(mask_arr == 2)

  submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
  submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed

  return submission_dict

  # Please notice that in this example we have a single prediction.
  # For the competition you have to provide segmentation for each of
  # the test images.

  # Finally, save the results into the submission.json file


In [ ]:
import json
import cv2

test_img_data_gen = ImageDataGenerator(rescale=1./255)
test_dir = os.path.join(dataset_dir, 'Test_Dev')
submission_dict = {}

for team in os.listdir(test_dir):
  team_test_dir = os.path.join(test_dir, team)

  for crop in os.listdir(team_test_dir):
    crop_test_dir = os.path.join(team_test_dir, crop)
    img_test_dir = os.path.join(crop_test_dir, 'Images')

    for file in os.listdir(img_test_dir):
      img = Image.open(os.path.join(img_test_dir, file))
      img_shape = img.size
      img = img.resize((1024, 768), Image.LANCZOS)
      img_arr = np.array(img)
      img_t = test_img_data_gen.get_random_transform(img_arr.shape, seed=SEED)
      img_arr = test_img_data_gen.apply_transform(img_arr, img_t)

      # Prediction of the images from the test generator, the label for each class is the argmax of the prediction
      out_sigmoid = model.predict(np.expand_dims(img_arr, axis=0))[0]
      out_sigmoid = cv2.resize(out_sigmoid, img_shape, interpolation=cv2.INTER_CUBIC)

      prediction_mask = tf.argmax(out_sigmoid, -1)
      img_name = os.path.splitext(file)[0]

      submission_dict = add_prediction(img_name, img_shape, prediction_mask, team, crop, submission_dict)

with open(os.path.join(base_dir, exp_dir, 'submission.json'), 'w') as f:
  json.dump(submission_dict, f)